In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import datetime
from datetime import datetime
import re

### cargar archivo 1

In [2]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padon_1 = pd.read_excel(ruta_archivo, engine='openpyxl')
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padon_1)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='//192.168.2.110/auditoria medica/sss salud PADRONES/padron/ftp novedades padron/2023/julio 2023/127406.txt' mode='r' encoding='cp1252'>


BadZipFile: File is not a zip file

In [8]:
print(padon_1['Fch_Nac'].head(44))

0              01/07/1974
1              25/02/1958
2              20/12/1997
3              12/08/2015
4              01/01/1979
5              28/10/2010
6              08/11/1977
7              25/01/2006
8              22/08/1990
9              23/01/1981
10             15/09/1997
11             05/11/2015
12             25/07/1983
13             14/12/1965
14             24/03/2003
15             15/02/1987
16             09/09/2015
17             12/12/2008
18             26/01/2007
19             06/11/1976
20             22/02/1957
21             28/06/1982
22             16/05/1979
23             20/09/1976
24             13/03/2005
25             04/07/1967
26             27/03/1986
27             13/10/1984
28             18/10/1972
29             16/01/1987
30             22/03/1983
31             28/07/1977
32             26/03/2000
33             10/10/1989
34             09/09/1991
35             21/12/1996
36             19/09/1977
37             20/03/1979
38          

### cargar archivo 2

In [3]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron_2 = pd.read_excel(ruta_archivo, engine='xlrd')
            padron_2 = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron_2)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")


<_io.TextIOWrapper name='C:/Users/zickd/Downloads/meplife/20729.csv' mode='r' encoding='cp1252'>
       Nro. OS      Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7            Desregulado     SI             26407599   
1            7            Desregulado     SI             26407599   
2            6               Afiliado     SI             30694069   
3           10            Desregulado     SI             31403090   
4            1  Monotributista social     NO             36821087   
...        ...                    ...    ...                  ...   
15523        4            Desregulado     SI             21601831   
15524        4            Desregulado     SI             21601831   
15525        4            Desregulado     SI             21601831   
15526        4            Desregulado     SI             21601831   
15527        1               Afiliado     SI             36602782   

         Nro. Afiliado Parentesco con el TITULAR Apellido Afiliado  \
0   

### Normalizar DNI archivo convenio

In [4]:

#armar el camp DNI juntando las los campos cuil titular y cuil familiar
padon_1['dni']=padon_1['CUIL_Familiar'].fillna(padon_1['CUIL_Titular'])
#limpiar el nuevo campo dni dejando solo los numeros 
padon_1['dni']=padon_1['dni'].apply(lambda x: re.sub('[^0-9]', '', str(x)))
#dejar solo los numreros eliminar caracteres 
padon_1['dni2']=padon_1['dni'].apply(lambda x: x[2:10] if len(x)>= 8 else '0')
#extraer el dni del cuil y dejar vacios los q son menores  a 8
padon_1['dni2']=padon_1['dni2'].astype(int)
#pasar el campo a int 
padon_1['Fch_Nac']=padon_1['Fch_Nac'].astype(str).str.zfill(8)
#padon_1['Fch_Nac']=padon_1['Fch_Nac'].astype(str)
#convertir fecha. 
print(padon_1['Fch_Nac'])
#padon_1=funciones.transformar_fecha(padon_1, 'Fch_Nac')

#Función para normalizar la fecha
def normalizar_fecha(fecha):
    fecha_normalizada = datetime.strptime(fecha, "%d%m%Y").strftime("%d/%m/%Y")
    return fecha_normalizada

#Aplicar la función a la columna "fech_nacimiento"
padon_1['Fch_Nac'] = padon_1['Fch_Nac'].apply(normalizar_fecha)
padon_1['Fch_Nac'] = padon_1['Fch_Nac'].astype(str)
padon_1[['provincia2','localidad']] = padon_1['Localidad'].str.split('-',expand = True, n=1)
print(padon_1.info(10))


padon_1.to_excel(r"C:\Users\zickd\Downloads\meplife\padon1.xlsx", engine='openpyxl')

0       01071974
1       25021958
2       20121997
3       12082015
4       01011979
          ...   
1682    17051992
1683    04051986
1684    08011965
1685    03071962
1686    18051985
Name: Fch_Nac, Length: 1687, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687 entries, 0 to 1686
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Codigo             1687 non-null   int64  
 1   CUIT_Empleador     1687 non-null   int64  
 2   CUIL_Titular       1687 non-null   int64  
 3   Parentesco         1687 non-null   int64  
 4   CUIL_Familiar      1687 non-null   int64  
 5   Tipo_Doc           1687 non-null   object 
 6   Nro_Doc            1687 non-null   int64  
 7   Apellido           1687 non-null   object 
 8   Nombre             1687 non-null   object 
 9   Sexo               1687 non-null   object 
 10  Estado_Civil       1687 non-null   int64  
 11  Fch_Nac            1687 non-null  

### Errores

In [10]:


condiciones = [
    lambda row: (row['Parentesco'] != 1 and row['Parentesco'] != 6 and row['Parentesco'] != 0) and row['dni2'] >999999 and (pd.to_datetime('today') - pd.to_datetime(row['Fch_Nac'], format='%d/%m/%Y')).days // 365.25 > 25,
    lambda row: row['dni2'] is None or len(str(row['dni2'])) < 8,
    lambda row: row['Fch_Nac'] is None or (pd.to_datetime('today') - pd.to_datetime(row['Fch_Nac'], format='%d/%m/%Y')).days // 365.25 > 80,
    lambda row: row['localidad'] is None
]
mensajes = [
    'familiar mayor de edad',
    'Error en DNI',
    'Error en la edad',
    'localidad vacía'
]

def generar_columnas_error(row):
    errores = {}
    for i in range(len(condiciones)):
        if condiciones[i](row):
            columna_error = f'error{i+1}'
            errores[columna_error] = mensajes[i]
    if errores:
        return pd.Series(errores)
    else:
        return pd.Series()

df_errores = padon_1.apply(generar_columnas_error, axis=1)
#print(df_errores.head(20))
df_nuevo = pd.concat([padon_1, df_errores],axis=1)
df_sin_errorv1 = df_nuevo[df_errores.any(axis=1)]
df_sin_errorv1.to_excel(r'C:\Users\zickd\Downloads\meplife\sin_erroresv1.xlsx', engine='openpyxl', index=False)

#df_sin_error = df_nuevo[df_nuevo['error1'].notna() | df_nuevo['error2'].notna() | df_nuevo['error3'].notna()]
df_con_error = df_nuevo[df_nuevo.get('error1', pd.Series()).notna() | df_nuevo.get('error2', pd.Series()).notna() | df_nuevo.get('error3', pd.Series()).notna() | df_nuevo.get('error4', pd.Series()).notna()]

df_con_error.to_excel(r'C:\Users\zickd\Downloads\meplife\con_errores.xlsx', engine='openpyxl', index=False)

#df_sin_errores = df_nuevo[df_nuevo['error1'].isnull() & df_nuevo['error2'].isnull() & df_nuevo['error3'].isnull()]

df_sin_errores = df_nuevo[~(df_nuevo.get('error1', pd.Series()).notna() & df_nuevo.get('error2', pd.Series()).notna() & df_nuevo.get('error3', pd.Series()).notna() & df_nuevo.get('error4', pd.Series()).notna())]



#df_errores.to_excel(r'C:\Users\zickd\Downloads\sin_errores.xlsx', engine='openpyxl', index=False)

df_sin_errores.to_excel(r'C:\Users\zickd\Downloads\meplife\sin_errores.xlsx', engine='openpyxl', index=False)


C:\Users\zickd\AppData\Local\Temp\ipykernel_3380\2568843314.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\zickd\AppData\Local\Temp\ipykernel_3380\2568843314.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\zickd\AppData\Local\Temp\ipykernel_3380\2568843314.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\Users\zickd\AppData\Local\Temp\ipykernel_3380\2568843314.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  return pd.Series()
C:\U

### Merge archivos x DNI

In [ ]:
cruce_tis=pd.merge(padon_1, padron_2, left_on='dni2', right_on='Nro. Documento', how='left', indicator=True)
#cruce_tis.to_excel(r'C:\Users\zickd\Downloads\merge.xlsx', engine='openpyxl', index=False)

cruce_tis_filtro=cruce_tis[cruce_tis['_merge']=='left_only']

#print(cruce_tis_filtro.head(5))

### exportar el cruce

In [ ]:
save_path = filedialog.askdirectory()
#uso iloc por si las columnas tienen el mismo nombre, necesito la primeras 20 columnas
cruce_tis_filtro.iloc[:, :28].to_excel(save_path + '\cruce_meplife.xlsx', engine='openpyxl' ,index = False )



In [3]:
fecha= '01/01/1974'

fecha_correg= (pd.to_datetime('today') - pd.to_datetime(padon_1['Fch_Nac'])).days // 365.25 > 50

edad= (pd.to_datetime('today') - pd.to_datetime(fecha)).days // 365.25

print(fecha_correg)


C:\Users\zickd\AppData\Local\Temp\ipykernel_26056\2035666616.py:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  fecha_correg= (pd.to_datetime('today') - pd.to_datetime(padon_1['Fch_Nac'])).days // 365.25 > 50


AttributeError: 'Series' object has no attribute 'days'